In [116]:
import pandas as pd
import networkx as nx
df = pd.read_csv('new_ds.csv')

/var/folders/q8/7rdzd_tn4vj2f9mgvf75cv0h0000gn/T/ipykernel_6153/2820079052.py:3: DtypeWarning: Columns (11,17,18,19,20,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('new_ds.csv')


In [117]:
df.dropna(subset=['client_inn',
'client_snils',
'contact_phone',
'contact_email'],inplace=True)
df.shape

(513537, 47)

In [118]:
df.columns

Index(['client_id', 'client_first_name', 'client_middle_name',
       'client_last_name', 'client_fio_full', 'client_bday', 'client_bplace',
       'client_cityzen', 'client_resident_cd', 'client_gender',
       'client_marital_cd', 'client_graduate', 'client_child_cnt',
       'client_mil_cd', 'client_zagran_cd', 'client_inn', 'client_snils',
       'client_vip_cd', 'contact_vc', 'contact_tg', 'contact_other',
       'contact_email', 'contact_phone', 'addr_region', 'addr_country',
       'addr_zip', 'addr_street', 'addr_house', 'addr_body', 'addr_flat',
       'addr_area', 'addr_loc', 'addr_city', 'addr_reg_dt', 'addr_str',
       'fin_rating', 'fin_loan_limit', 'fin_loan_value', 'fin_loan_debt',
       'fin_loan_percent', 'fin_loan_begin_dt', 'fin_loan_end_dt',
       'stream_favorite_show', 'stream_duration', 'create_date', 'update_date',
       'source_cd'],
      dtype='object')

In [119]:
G = nx.Graph()

G.add_nodes_from(df['client_id'])

features = ['client_inn',
'client_snils',
'contact_phone',
'contact_email',
'contact_vc',
'contact_tg']

for feature in features:
    grouped = df.groupby(feature)['client_id'].apply(list)
    for group in grouped:
        if len(group) > 1:
            for i in range(len(group)):
                for j in range(i + 1, len(group)):
                    G.add_edge(group[i], group[j])
                    
components = list(nx.connected_components(G))



In [120]:
cnt = 0
for i in components:
    cnt += len(i)
print(cnt)

503373


In [121]:
client_group = {}
for idx, component in enumerate(components):
    for client_id in component:
        client_group[client_id] = idx


df['group_id'] = df['client_id'].map(client_group)

print(df['group_id'].nunique())

df.to_csv('groupped.csv',index=False)

242292


In [122]:
result = df.groupby('group_id').agg(list).reset_index()

In [123]:
result.to_csv('inn_graph.csv',index=False)

In [124]:
result

,group_id,client_id,client_first_name,client_middle_name,client_last_name,client_fio_full,client_bday,client_bplace,client_cityzen,client_resident_cd,...,fin_loan_value,fin_loan_debt,fin_loan_percent,fin_loan_begin_dt,fin_loan_end_dt,stream_favorite_show,stream_duration,create_date,update_date,source_cd
0,0,"[20712224, 22252901]","[ШЕРИН, Шерин]","[МАДРИДОВИЧ, Мадридович]","[КОСЯНОК, Косянок]","[КОСЯНОК ШЕРИН МАДРИДОВИЧ, Косянок Шерин Мадри...","[1929-06-29, 1929-06-29]",[С.ВАСИЛЬЕВКА ОКОНЕШНИКОВСКОГО Р-НА ОМСКОЙ ОБЛ...,"[RUS, RUS]","[Д, Д]",...,"[123000.0, 123000.0]","[112143.0, 112143.0]","[4.0, 4.0]","[2022-05-28, 2022-05-28]","[2022-12-24, 2022-12-24]","[nan, nan]","[nan, nan]","[2023-09-22 05:00:12.444, 2024-02-09 03:21:23....","[2024-09-20 09:58:55.608, 2024-02-09 03:21:23....","[Bank2, Bank]"
1,1,"[696309814, 551502803, 675691748]","[БАТУРДИН, БАТУРДИН, Батурдин]","[БАТО-МУНКУЕВИЧ, БАТО-МУНКУЕВИЧ, Бато-Мункуевич]","[ГУМЕННИК, ГУМЕННИК, Гуменник]","[ГУМЕННИК БАТУРДИН БАТО-МУНКУЕВИЧ, ГУМЕННИК БА...","[1997-06-17, 1997-06-17, 1997-06-17]","[ПГТ.ГОРНЫЙ ТОГУЧИНСКОГО Р-НА НСО, ПГТ.ГОРНЫЙ ...","[RUS, RUS, RUS]","[Д, Д, Д]",...,"[947000.0, 947000.0, 947000.0]","[692768.0, 692768.0, 692768.0]","[33.0, 33.0, 33.0]","[2018-01-01, 2018-01-01, 2018-01-01]","[2018-09-28, 2018-09-28, 2018-09-28]","[nan, nan, nan]","[nan, nan, nan]","[2022-09-19 09:24:02.917, 2021-10-03 01:57:20....","[2024-09-20 09:50:27.704, 2021-10-03 01:57:20....","[Bank2, Bank2, Bank]"
2,2,[575615912],[ТЮЛЮБАЙ],[ЯМИЛЬЕВИЧ],[СИВЧИКОВ],[СИВЧИКОВ ТЮЛЮБАЙ ЯМИЛЬЕВИЧ],[1936-01-01],[С. ИВАНКОВО АЛЬМЕНЕВСКОГО Р-НА КУРГАНСКОЙ ОБЛ.],[RUS],[Д],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[2022-12-28 01:31:21.859],[2024-09-20 09:47:50.668],[Bank2]
3,3,"[669104926, 674053892, 636364695]","[ЗАЙНОБИДИН, ЗАЙНОБИДИН, Зайнобидин]","[ДУНДАРОВИЧ, ДУНДАРОВИЧ, Дундарович]","[САНДИН, САНДИН, Сандин]","[САНДИН ЗАЙНОБИДИН ДУНДАРОВИЧ, САНДИН ЗАЙНОБИД...","[1924-12-03, 1924-12-03, 1924-12-03]",[ДЕР. ВОРОНЖА СЕРЫШЕВСКОГО РАЙОНА АМУРСКОЙ ОБЛ...,"[RUS, RUS, RUS]","[Д, Д, Д]",...,"[27472000.0, 27472000.0, 27472000.0]","[5107986.0, 5107986.0, 5107986.0]","[12.0, 12.0, 12.0]","[2017-02-10, 2017-02-10, 2017-02-10]","[2044-04-04, 2044-04-04, 2044-04-04]","[nan, nan, nan]","[nan, nan, nan]","[2024-02-23 15:08:28.554, 2020-11-12 16:19:58....","[2024-09-20 09:42:02.613, 2020-11-12 16:19:58....","[Bank2, Bank2, Bank]"
4,4,"[71407443, 764828825, 960607140]","[МУРОДЖОН, МУРОДЖОН, Муроджон]","[ОКРАЕВИЧ, ОКРАЕВИЧ, Окраевич]","[БУРЛЮК, БУРЛЮК, Бурлюк]","[БУРЛЮК МУРОДЖОН ОКРАЕВИЧ, БУРЛЮК МУРОДЖОН ОКР...","[1929-12-12, 1929-12-12, 1929-12-12]","[С.АПАЛИХА ХВАЛЫНСКОГО Р-НА САРАТОВСКОЙ ОБЛ., ...","[RUS, RUS, RUS]","[Д, Д, Д]",...,"[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[2024-05-10 18:06:15.059, 2018-07-21 08:46:42....","[2024-09-20 09:17:08.415, 2023-01-25 13:48:44....","[Bank2, Bank2, Bank]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242287,242287,[832207617],[Зайнагатдин],[Гакильевич],[Ротс],[Ротс Зайнагатдин Гакильевич],[1929-09-26],[Д. ОДИНА ИСЕТСКОГО РАЙОНА ТЮМЕНСКОЙ ОБЛАСТИ],[RUS],[Д],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[2010-09-02 04:16:00.806],[2010-09-02 04:16:00.806],[Bank]
242288,242288,[663704518],[Майсалбек],[Дибирович],[Красотский],[Красотский Майсалбек Дибирович],[1952-10-02],[гор. Бататйск РО],[RUS],[Д],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[2010-09-01 23:34:36.582],[2010-09-01 23:34:36.582],[Bank]
242289,242289,[555690145],[Мустамир],[Женоидович],[Уродков],[Уродков Мустамир Женоидович],[1932-05-22],[КИРПИЧНЫЙ ЗАВОД ОРДЖОНИКИДЗЕВСКОГО Р-НА ТАШКЕ...,[RUS],[Д],...,[nan],[nan],[nan],[nan],[nan],[nan],[nan],[2010-09-01 22:43:34.407],[2010-09-01 22:43:34.407],[Bank]
242290,242290,[383580318],[Ляна],[Хаиргалиевна],[Заречняк],[Заречняк Ляна Хаиргалиевна],[nan],[ГОР.БРЯНКА ЛУГАНСКОЙ ОБЛ.РЕСПУБЛИКИ УКРАИНА],[RUS],[Д],...,[2284000.0],[594890.0],[18.0],[2017

In [125]:
cnt=0
for i in  result['client_id']:
    cnt += len(i)
print(cnt)

513537


In [126]:
import math

def first_el(x):
    for el in x:
        if el is not None and not (isinstance(el, float) and math.isnan(el)) and el != "":
            return el
    return ''

import math
from collections import Counter

def choose_el(x):
    filt_el = [
        el for el in x 
        if el is not None and not (isinstance(el, float) and math.isnan(el)) and el != ""
    ]
    
    if not filt_el:
        return ''

    most_common = Counter(filt_el).most_common(1)
    return most_common[0][0] if most_common else ''

In [127]:
first = ['client_id', 'client_child_cnt',
       'client_vip_cd', 'contact_vc', 'contact_tg',
       'contact_email', 'contact_phone',
       'addr_street', 'addr_house', 'addr_body', 'addr_flat',
       'addr_area', 'addr_loc', 'addr_city', 'addr_reg_dt',
       'fin_rating', 'fin_loan_limit', 'fin_loan_value', 'fin_loan_debt',
       'fin_loan_percent', 'fin_loan_begin_dt', 'fin_loan_end_dt',
       'create_date', 'update_date']



choose = ['client_first_name', 'client_middle_name',
'client_last_name', 'client_fio_full', 'client_bday',
'client_bplace', 'client_cityzen', 'client_resident_cd',
'client_gender', 'client_marital_cd', 'client_graduate',
'client_mil_cd', 'client_zagran_cd', 'client_inn',
'client_snils', 'contact_other', 'addr_region',
'addr_country', 'addr_zip', 'addr_str',
'stream_favorite_show', 'stream_duration', 'source_cd']

print(len(first),len(choose))

24 23


In [128]:
for el in first:
    result[el] = result[el].apply(first_el)
    
for el in choose:
    result[el] = result[el].apply(choose_el)

result.to_csv('ans.csv',index=False)